# 🎵 Модуль анализа аудио (YAMNet)

## Распознавание звуков природы и голосов птиц

<a id='4'></a>
## 4. 🎵 Модуль анализа аудио (YAMNet)

### О модели YAMNet

**YAMNet (Yet Another Mobile Network)** - предобученная модель от Google для классификации звуков.

**Характеристики:**
- ✅ Обучена на датасете AudioSet (2+ млн аудиоклипов)
- ✅ 521 класс звуков (речь, музыка, звуки природы, животных и т.д.)
- ✅ Входные данные: 16 кГц моно аудио
- ✅ Быстрая инференс (реальное время)

**Классы звуков природы:**
- 🐦 Голоса птиц (Bird, Bird vocalization, Chirp, Tweet и т.д.)
- 🐸 Звуки лягушек
- 🦗 Насекомые (Cricket, Insect)
- 🌊 Природные явления (Wind, Rain, Thunder, Stream)
- И многое другое...

In [ ]:
# Импорт необходимых библиотек (если ещё не импортированы)
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

print("✅ Библиотеки импортированы!")

### 4.1 Загрузка модели YAMNet

In [ ]:
print("📥 Загрузка модели YAMNet из TensorFlow Hub...")
print("⚠️  При первом запуске модель будет скачана (может занять несколько минут)")

# Загрузка модели YAMNet
yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1')

print("✅ Модель YAMNet успешно загружена!")

# Загрузка названий классов
class_names_url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/audioset/yamnet/yamnet_class_map.csv'
class_map_df = pd.read_csv(class_names_url)
class_names = class_map_df['display_name'].tolist()

print(f"📊 Модель обучена на {len(class_names)} классах звуков")

# Классы звуков, связанных с природой и животными
NATURE_KEYWORDS = [
    'bird', 'chirp', 'tweet', 'squawk', 'pigeon', 'crow', 'owl',
    'animal', 'insect', 'cricket', 'frog', 'toad',
    'wind', 'rain', 'thunder', 'stream', 'water', 'ocean', 'wave',
    'rustle', 'forest'
]

# Поиск индексов природных звуков
nature_classes = {}
for idx, name in enumerate(class_names):
    name_lower = name.lower()
    for keyword in NATURE_KEYWORDS:
        if keyword in name_lower:
            nature_classes[idx] = name
            break

print(f"\n🌿 Найдено {len(nature_classes)} классов природных звуков")
print("\nПримеры классов:")
for idx, (class_idx, class_name) in enumerate(list(nature_classes.items())[:10]):
    print(f"   {class_idx:3d}: {class_name}")

### 4.2 Класс для анализа аудио

In [ ]:
class AudioAnalyzer:
    """
    Класс для анализа аудиозаписей с использованием YAMNet
    
    Функциональность:
    - Классификация звуков
    - Обнаружение голосов птиц и звуков природы
    - Спектральный анализ
    - Визуализация результатов
    - Формирование статистики
    """
    
    def __init__(self, model, class_names, nature_classes, sample_rate=16000):
        """
        Инициализация анализатора
        
        Args:
            model: Загруженная модель YAMNet
            class_names: Список названий классов
            nature_classes: Словарь природных классов
            sample_rate: Частота дискретизации (Hz)
        """
        self.model = model
        self.class_names = class_names
        self.nature_classes = nature_classes
        self.sample_rate = sample_rate
        self.detections = []
        self.statistics = {}
    
    def load_audio(self, audio_path):
        """
        Загрузка аудиофайла
        
        Args:
            audio_path: Путь к аудиофайлу
            
        Returns:
            waveform: Аудиосигнал
            sr: Частота дискретизации
        """
        # Загрузка аудио с помощью librosa
        waveform, sr = librosa.load(audio_path, sr=self.sample_rate, mono=True)
        
        # Нормализация амплитуды
        waveform = waveform / np.max(np.abs(waveform))
        
        return waveform, sr
    
    def classify_audio(self, waveform, top_k=5, confidence_threshold=0.1):
        """
        Классификация аудио с помощью YAMNet
        
        Args:
            waveform: Аудиосигнал
            top_k: Количество топ предсказаний
            confidence_threshold: Минимальный порог уверенности
            
        Returns:
            results: Результаты классификации
        """
        # Запуск модели
        scores, embeddings, spectrogram = self.model(waveform)
        
        # scores имеет размерность [N, 521], где N - количество сегментов
        # Усреднение предсказаний по всем сегментам
        scores_mean = tf.reduce_mean(scores, axis=0).numpy()
        
        # Топ-K предсказаний
        top_indices = np.argsort(scores_mean)[::-1][:top_k]
        
        results = []
        for idx in top_indices:
            score = scores_mean[idx]
            if score >= confidence_threshold:
                results.append({
                    'class_id': int(idx),
                    'class_name': self.class_names[idx],
                    'confidence': float(score),
                    'is_nature': idx in self.nature_classes
                })
        
        return results, scores, spectrogram
    
    def analyze_audio(self, audio_path, confidence_threshold=0.2):
        """
        Полный анализ аудиофайла
        
        Args:
            audio_path: Путь к аудиофайлу
            confidence_threshold: Порог уверенности
            
        Returns:
            analysis_results: Результаты анализа
        """
        print(f"🎵 Начало анализа аудио: {audio_path}")
        
        # Загрузка аудио
        waveform, sr = self.load_audio(audio_path)
        duration = len(waveform) / sr
        
        print(f"⏱️  Длительность: {duration:.2f} сек")
        print(f"📊 Частота дискретизации: {sr} Hz")
        print(f"📈 Семплов: {len(waveform)}")
        
        # Классификация
        print("\n🔍 Классификация звуков...")
        classifications, scores, spectrogram = self.classify_audio(
            waveform,
            top_k=10,
            confidence_threshold=confidence_threshold
        )
        
        # Фильтрация природных звуков
        nature_sounds = [c for c in classifications if c['is_nature']]
        
        # Формирование результатов
        analysis_results = {
            'audio_path': str(audio_path),
            'duration': duration,
            'sample_rate': sr,
            'waveform': waveform,
            'classifications': classifications,
            'nature_sounds': nature_sounds,
            'spectrogram': spectrogram.numpy(),
            'scores': scores.numpy()
        }
        
        print(f"\n✅ Анализ завершён!")
        print(f"📊 Найдено классов: {len(classifications)}")
        print(f"🌿 Природных звуков: {len(nature_sounds)}")
        
        return analysis_results
    
    def plot_waveform(self, waveform, sr, title="Аудиосигнал"):
        """
        Визуализация аудиосигнала
        
        Args:
            waveform: Аудиосигнал
            sr: Частота дискретизации
            title: Заголовок графика
        """
        plt.figure(figsize=(15, 4))
        time = np.linspace(0, len(waveform) / sr, num=len(waveform))
        plt.plot(time, waveform, linewidth=0.5)
        plt.xlabel('Время (сек)')
        plt.ylabel('Амплитуда')
        plt.title(title)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    
    def plot_spectrogram(self, waveform, sr, title="Спектрограмма"):
        """
        Визуализация спектрограммы
        
        Args:
            waveform: Аудиосигнал
            sr: Частота дискретизации
            title: Заголовок графика
        """
        plt.figure(figsize=(15, 5))
        D = librosa.amplitude_to_db(np.abs(librosa.stft(waveform)), ref=np.max)
        librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='hz')
        plt.colorbar(format='%+2.0f dB')
        plt.title(title)
        plt.tight_layout()
        plt.show()
    
    def plot_classifications(self, classifications, title="Классификация звуков"):
        """
        Визуализация результатов классификации
        
        Args:
            classifications: Результаты классификации
            title: Заголовок графика
        """
        if not classifications:
            print("⚠️  Нет данных для визуализации")
            return
        
        df = pd.DataFrame(classifications)
        df = df.sort_values('confidence', ascending=True)
        
        plt.figure(figsize=(12, 6))
        colors = ['green' if is_nat else 'gray' for is_nat in df['is_nature']]
        plt.barh(df['class_name'], df['confidence'], color=colors)
        plt.xlabel('Уверенность')
        plt.title(title)
        plt.tight_layout()
        plt.show()

print("✅ Класс AudioAnalyzer создан!")

### 4.3 Создание демо-аудио для тестирования

In [ ]:
def create_demo_audio(output_path='data/audio/demo_nature.wav', duration=5, sr=16000):
    """
    Создание демонстрационного аудиофайла с синтетическими звуками
    
    Args:
        output_path: Путь для сохранения
        duration: Длительность в секундах
        sr: Частота дискретизации
    """
    import soundfile as sf
    
    print("🎨 Создание демонстрационного аудио...")
    
    t = np.linspace(0, duration, sr * duration)
    
    # Имитация звуков природы (комбинация частот)
    # "Щебетание птицы" - высокие частоты с модуляцией
    bird_chirp = 0.3 * np.sin(2 * np.pi * 2000 * t) * (1 + 0.5 * np.sin(2 * np.pi * 5 * t))
    
    # "Шум ветра" - низкие частоты
    wind = 0.1 * np.random.normal(0, 1, len(t))
    wind_filtered = librosa.effects.preemphasis(wind)
    
    # Комбинация звуков
    audio = bird_chirp + wind_filtered
    
    # Нормализация
    audio = audio / np.max(np.abs(audio))
    
    # Сохранение
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    sf.write(output_path, audio, sr)
    
    print(f"✅ Демо-аудио создано: {output_path}")
    print(f"   Длительность: {duration}с, Частота: {sr} Hz")

# Создание демо-аудио
demo_audio_path = 'data/audio/demo_nature.wav'
if not Path(demo_audio_path).exists():
    create_demo_audio(demo_audio_path, duration=10, sr=16000)
else:
    print(f"ℹ️  Демо-аудио уже существует: {demo_audio_path}")

### 4.4 Запуск анализа аудио

In [ ]:
# Инициализация анализатора
audio_analyzer = AudioAnalyzer(
    model=yamnet_model,
    class_names=class_names,
    nature_classes=nature_classes,
    sample_rate=16000
)

# Запуск анализа
audio_results = audio_analyzer.analyze_audio(
    audio_path=demo_audio_path,
    confidence_threshold=0.1
)

# Вывод результатов
print("\n" + "="*60)
print("🎵 РЕЗУЛЬТАТЫ АНАЛИЗА АУДИО")
print("="*60)

print(f"\n📻 Аудио: {audio_results['audio_path']}")
print(f"⏱️  Длительность: {audio_results['duration']:.2f} сек")
print(f"📊 Частота: {audio_results['sample_rate']} Hz")

print("\n🔝 ТОП-10 классов:")
for i, cls in enumerate(audio_results['classifications'][:10], 1):
    nature_mark = "🌿" if cls['is_nature'] else "  "
    print(f"{i:2d}. {nature_mark} {cls['class_name']:30s} ({cls['confidence']:.3f})")

print(f"\n🌿 Природных звуков обнаружено: {len(audio_results['nature_sounds'])}")
if audio_results['nature_sounds']:
    print("\nПриродные звуки:")
    for i, cls in enumerate(audio_results['nature_sounds'], 1):
        print(f"{i}. {cls['class_name']:30s} ({cls['confidence']:.3f})")

### 4.5 Визуализация аудио

In [ ]:
# График аудиосигнала
audio_analyzer.plot_waveform(
    audio_results['waveform'],
    audio_results['sample_rate'],
    title="Аудиосигнал - Звуки природы"
)

In [ ]:
# Спектрограмма
audio_analyzer.plot_spectrogram(
    audio_results['waveform'],
    audio_results['sample_rate'],
    title="Спектрограмма - Звуки природы"
)

In [ ]:
# График классификаций
audio_analyzer.plot_classifications(
    audio_results['classifications'],
    title="Классификация звуков (зелёные - природные)"
)